In [ ]:
import hpelm
from sklearn.metrics import precision_recall_fscore_support

from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam

from keras.layers.core import Dense, Activation, Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Layout, Figure, Marker
import tensorflow as tf
from matplotlib import pyplot as plt

CNN_EPOCH =30
image_shape=(224,224,3)
NUM_CLASS = 2
from keras.callbacks import ReduceLROnPlateau
import time


In [ ]:
def dataload():
    train_data=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,)
    training_data= train_data.flow_from_directory("Dehazed_Images_3400/Dehazed_Images/train",target_size=(224,224),class_mode='binary' )
    training_data

    test_data=ImageDataGenerator(rescale=1/255)
    testing_data=test_data.flow_from_directory("Dehazed_Images_3400/Dehazed_Images/test",target_size=(224,224),class_mode='binary')
    return training_data,testing_data

In [ ]:
def cnn_generate(training_data,testing_data):
   
    np.random.seed(1000)
    training_data_oh = np_utils.to_categorical(training_data.classes, NUM_CLASS)
    model=Sequential()

    #1
    model.add(Conv2D(filters=96,input_shape=image_shape,kernel_size=(11,11),strides=(4,4),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))


    #2
    model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))

    #3
    model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #4
    model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #5
    model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', ))

    model.add(Flatten())
 
    #Ist fc layer 
    model.add(Dense(4096,input_shape=(224,224,3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.4))
    
    #output layer
    model.add(Dense(1))    
    model.add(BatchNormalization())
    model.add(Activation("tanh"))
    
    adam = Adam(learning_rate=0.001)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    lrr= ReduceLROnPlateau( monitor='val_accuracy',   factor=.01,   patience=3,  min_lr=1e-5) 
    
  

    t0=time.time()
    history=model.fit(training_data,epochs=CNN_EPOCH,validation_data=testing_data,validation_steps=2,callbacks = [lrr])
    print("Training time :" , time.time()-t0)
    #accuracy vs validation accuracy
    p1=plt
    p1.plot(history.history['accuracy'])
    p1.plot(history.history['val_accuracy'])
    p1.title('model accuracy')
    p1.ylabel('accuracy')
    p1.xlabel('epoch')
    p1.legend(['train', 'test'], loc='upper left')
    p1.show()


    p2=plt
    #training loss vs validation loss
    p2.plot(history.history['loss'])
    p2.plot(history.history['val_loss'])
    p2.title('model loss')
    p2.ylabel('loss')
    p2.xlabel('epoch')
    p2.legend(['train', 'test'], loc='upper left')
    p2.show()
    return model



In [ ]:
def hidden_layer_generate(cnn_model,training_data):
    layer_name = 'flatten'
    hidden_layer_model = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer(layer_name).output)

    cnn_train_result = hidden_layer_model.predict(training_data)
    #print(cnn_train_result)
    return hidden_layer_model, cnn_train_result

In [ ]:
def elm_model_generate(data_train, training_data):
    NUM_CLASS=2
    ELM_HIDDEN_NEURONS=256
    target_train_oh = np_utils.to_categorical(training_data.classes, NUM_CLASS)
    
    elm_model = hpelm.elm.ELM(data_train.shape[1], NUM_CLASS)
    elm_model.add_neurons(ELM_HIDDEN_NEURONS, func='sigm')
    #print(data_train,target_train_oh)
    elm_model.train(data_train, target_train_oh, 'c')
    
    return elm_model

In [ ]:
from sklearn import metrics
import seaborn as sns
def cnn_elm_evaluation(cnn_part, elm_part, data_test, testing_data, file_name="plot_result.html"):

    """
    :param cnn_part: CNN Model
    :param elm_part: ELM Model
    :return: Result Score
    """

    target_test_oh = np_utils.to_categorical(testing_data.classes, NUM_CLASS)

    cnn_result = cnn_part.predict(data_test)
    elm_result = elm_part.predict(cnn_result)

    elm_result_class = np.array([np.argmax(r) for r in elm_result])
    #print(elm_result_class)
    
    confusion = elm_part.confusion(target_test_oh, elm_result)
    print(confusion)

    accuracy = (confusion[0][0]+confusion[1][1])/(confusion[0][0]+confusion[0][1]+confusion[1][0]+confusion[1][1])
   
    
    Sensitivity = confusion[0][0]/(confusion[0][0]+confusion[1][1])
    Specificity =confusion[1][1]/(confusion[0][1]+confusion[1][1])
    fpr = 1-Specificity
    #cm_display=metrics.ConfusionMatrixDisplay(confusion,display_labels=[False,True])
    #cm_display.plot()
    print("accuracy =" ,accuracy)
    print("Sensitivity=",Sensitivity)
    print("Specificity=" ,Specificity)
    print("fpr=",fpr)
    
    error=elm_part.error(target_test_oh,elm_result)
    print("error=",error)

    
    # Confusion Matrix Plot
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
                confusion.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                        confusion.flatten()/np.sum(confusion)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
            zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(confusion, annot=labels, fmt='',xticklabels=['busy','free'],yticklabels=['busy','free'],)

    
    
    precision, recall, fscore, support = precision_recall_fscore_support(testing_data.classes, elm_result_class)
    
    print(precision, recall, fscore, support)

    return precision, recall, fscore, support

In [ ]:
def plot_precision_recall_fscore(precision, recall, fscore, filename='new_result.html'):
    print("precision=",precision,"recall=",recall,"fscore=",fscore)
    avg_data = np.array([
        np.mean(precision),
        np.mean(recall),
        np.mean(fscore)
    ])

    panda_avg = pd.DataFrame(avg_data).round(3)

    data = np.array(panda_avg.values).T[0]

    print(avg_data)
    print(panda_avg)
    print(data)
    
    trace = [plotly.graph_objs.Bar(
        x=['precision', 'recall', 'fscore'],
        y=data,
        marker=Marker(color='#04b486'),
        text=data,
        textposition='auto',
        textfont=dict(color='black', size=20)
    )]

    layout = Layout(bargap=0.6, plot_bgcolor='#E6E6E6')
    plot_data = Figure(data=trace, layout=layout)
    plotly.offline.plot(plot_data, filename=filename)

In [ ]:
if __name__ == '__main__':

    training_data,testing_data=dataload()
    t1=time.time()    
    model=cnn_generate(training_data,testing_data)
    hidden_layer_model, cnn_train_result = hidden_layer_generate(model,training_data)
   
    elm_model = elm_model_generate(cnn_train_result, training_data)

    precision, recall, fscore, support = cnn_elm_evaluation(hidden_layer_model, elm_model, testing_data, testing_data)

    plot_precision_recall_fscore(precision, recall, fscore)
    print("total time",time.time()-t1)

    

In [ ]:

   
elm_model = elm_model_generate(cnn_train_result, training_data)

precision, recall, fscore, support = cnn_elm_evaluation(hidden_layer_model, elm_model, testing_data, testing_data)

plot_precision_recall_fscore(precision, recall, fscore)
print("total time",time.time()-t1)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Example data (replace with your actual data)
classifiers = ['Classifier1', 'Classifier2']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
folds = ['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5']

# Sample performance metrics (replace with your actual metrics)
data = {
    ('Classifier1', 'Accuracy'): [0.85, 0.82, 0.87, 0.78, 0.84],
    ('Classifier1', 'Precision'): [0.80, 0.77, 0.82, 0.75, 0.79],
    ('Classifier1', 'Recall'): [0.75, 0.72, 0.77, 0.70, 0.75],
    ('Classifier1', 'F1-score'): [0.78, 0.75, 0.80, 0.73, 0.78],
    ('Classifier2', 'Accuracy'): [0.80, 0.78, 0.82, 0.75, 0.79],
    ('Classifier2', 'Precision'): [0.78, 0.75, 0.79, 0.72, 0.78],
    ('Classifier2', 'Recall'): [0.72, 0.70, 0.74, 0.67, 0.72],
    ('Classifier2', 'F1-score'): [0.75, 0.73, 0.77, 0.70, 0.75],
    # Add data for other classifiers...
}

# Convert data to a DataFrame
df = pd.DataFrame(data)

# Calculate mean performance metrics across folds
mean_performance = df.mean(axis=1, level=1)

# Plot line graph
plt.figure(figsize=(10, 6))

for classifier in classifiers:
    for metric in metrics:
        plt.plot(folds, mean_performance[classifier][metric], label=f"{classifier} - {metric}")

plt.title('Mean Performance Metrics Across Folds')
plt.xlabel('Fold')
plt.ylabel('Mean Performance Metric')
plt.legend(loc='best')
plt.grid(True)
plt.show()
